# Deep Image Neural Net w/ CIFAR10

In [18]:
import torch
import torch.nn as nn
import torchvision.datasets
import torchvision.transforms.v2 as T
import matplotlib.pyplot as plt

toTensor = T.Compose([T.ToImage(), T.ToDtype(torch.float32, scale=True)])

train_and_valid_data = torchvision.datasets.CIFAR10(
    root="datasets", train=True, download=True, transform=toTensor
)
test_data = torchvision.datasets.CIFAR10(
    root="datasets", train=False, download=True, transform=toTensor
)

train_data, valid_data = torch.utils.data.random_split(
    train_and_valid_data, [45000, 5000]
)

In [15]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(3*32*32, 100), nn.SiLU(),
    nn.Linear(100,100), nn.SiLU(),
    nn.Linear(100,100), nn.SiLU(),
    nn.Linear(100,100), nn.SiLU(),
    nn.Linear(100,100), nn.SiLU(),
    nn.Linear(100,10)
)
for idx, module in model.named_modules():
    if isinstance(module, nn.Linear):
        nn.init.kaiming_uniform_(module.weight)
        nn.init.zeros_(module.bias)

device = "cuda"

In [25]:
def train(model, optimizer, criterion, train_loader, n_epochs):
    model.train()
    loss_epochs = []
    for epoch in range(1, n_epochs+1):
        total_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            total_loss+=loss.item()
            optimizer.step()
            optimizer.zero_grad()
        loss_epochs.append((epoch, total_loss/len(train_loader)))
    return loss_epochs

model.to(device)
lr = .003
optimizer = torch.optim.NAdam(model.parameters(), lr=lr)
xentropy = nn.CrossEntropyLoss()
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
losses = train(model, optimizer, xentropy, train_loader, 30)
plt.plot([x[0] for x in losses], [x[1] for x in losses])
plt.show()

KeyboardInterrupt: 